In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [ ]:
# 크롬 웹드라이버 실행 경로
path = 'chromedriver.exe'

# 크롤링할 주소 (네이버 세부설정까지 완료한 상태로)
target_url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2023.07.01&de=2023.12.31&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1009&nso=so%3Ar%2Cp%3Afrom20230701to20231231&is_sug_officeid=0&office_category=0&service_area=0"

# 크롬 드라이버 사용
service = Service(executable_path=path) # selenium 최근 버전은 이렇게
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# 드라이버에게 크롤링할 대상 알려주기
try:
    # 웹페이지 열기
    driver.get(target_url)

    # 초기 스크롤 높이 설정
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 아래까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 적응 시간
        time.sleep(0.5)  

        # 새로운 스크롤 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 만약 더 스크롤할게 없다면 정지
        last_height = new_height

    # 해당 화면 안에서 페이지 소스 받기
    page_source = driver.page_source

    # BeautifulSoup으로 parsing
    bs_obj = BeautifulSoup(page_source, 'html.parser')

    # a태그로 info에 해당하는 태그들 다 수집
    results = bs_obj.find_all('a', {'class': "info"})

    # 수집한 a 태그 개수 -> 절반이 네이버 뉴스
    print(f"Number of results found: {len(results)/2}")

finally:
    # 가상 화면 닫기
    driver.quit()

In [ ]:
# 기존 한국경제 링크랑 섞여서 네이버 뉴스꺼만 필터링
news_link = []
for tag in results:
    href = tag.get('href')
    if href:
        news_link.append(href)

news_link = [i for index, i in enumerate(news_link) if (index + 1) % 2 == 0]

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [ ]:
from tqdm import tqdm

all_results = dict()

# tqdm을 사용하여 진행 상황 표시
for i, link in enumerate(tqdm(news_link, desc="Processing links")):     
    # scraping하려는 웹페이지 주소를 get()에 전달
    driver.get(link)
    all_results[i] = dict()
    
    # 일시 추출하기
    date = driver.find_elements(By.CLASS_NAME, 'media_end_head_info_datestamp_bunch')
    date = date[0].text[2:]
    
    # 제목 추출하기
    title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
    title = title[0].text
    
    # 본문 추출하기
    body = driver.find_elements(By.ID, 'newsct_article')
    body = body[0].text.replace('\n','')
    
    # 결과 저장
    all_results[i]['link'] = link
    all_results[i]['date'] = date
    all_results[i]['title'] = title
    all_results[i]['body'] = body

# 드라이버 종료
driver.quit()

In [ ]:
news_df = pd.DataFrame(all_results).T

In [ ]:
# '오전'과 '오후'를 영어로 변환
news_df['date'] = news_df['date'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# date 열을 datetime 형식으로 변환
news_df['date'] = pd.to_datetime(news_df['date'], format='%Y.%m.%d. %p %I:%M')

# year, month, day, hour, minute 열 생성
news_df['year'] = news_df['date'].dt.year
news_df['month'] = news_df['date'].dt.month
news_df['day'] = news_df['date'].dt.day
news_df['hour'] = news_df['date'].dt.hour
news_df['minute'] = news_df['date'].dt.minute

In [ ]:
# 고유 인덱스 부여 (년도 + 6자리)
# 1이면 -> 000001
news_df.index = news_df.index + 1
news_df = news_df.reset_index()
news_df['index'] = news_df['index'].astype(str).str.zfill(6)
news_df['index'] = str(news_df['year'].unique()[0]) + news_df['index']

In [ ]:
news_df

In [ ]:
news_df.to_csv('2023_매일경제_전반기.csv', index=False, encoding='utf-8-sig')